In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/DS340W')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('ai_job_market_insights.csv')
df.head()

,Job_Title,Industry,Company_Size,Location,AI_Adoption_Level,Automation_Risk,Required_Skills,Salary_USD,Remote_Friendly,Job_Growth_Projection
0,1,2,2,1,2,0,9,111392.165243,1,1
1,4,7,0,6,2,0,5,93792.562466,0,0
2,0,7,0,6,2,0,9,107170.263069,1,1
3,7,6,2,0,1,0,6,93027.953758,0,1
4,1,2,2,8,1,1,3,87752.922171,1,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save the encoder for each column

# Define features and target variable
X = df.drop('Automation_Risk', axis=1)
y = df['Automation_Risk']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Verify the processed data structure
X_train.head(), y_train.head()


(     Job_Title  Industry  Company_Size  Location  AI_Adoption_Level  \
 5            9         0             0         5                  2   
 116          9         8             2         0                  1   
 45           7         4             0         6                  2   
 16           8         5             1         6                  0   
 462          9         5             2         4                  1   
 
      Required_Skills     Salary_USD  Remote_Friendly  Job_Growth_Projection  
 5                  1  102825.007867                0                      1  
 116                7   93313.178871                0                      0  
 45                 2   93086.342533                0                      1  
 16                 4   78401.259282                0                      0  
 462                3   85743.834770                0                      1  ,
 5      2
 116    1
 45     2
 16     2
 462    2
 Name: Automation_Risk, dtype: int64)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize the classifiers
dt_classifier = DecisionTreeClassifier(random_state=42)
rf_classifier = RandomForestClassifier(random_state=42)
lr_classifier = LogisticRegression(max_iter=200, random_state=42)

# Train and evaluate Decision Tree
dt_classifier.fit(X_train, y_train)
y_pred_dt = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred_dt)
dt_report = classification_report(y_test, y_pred_dt, zero_division=0)

# Train and evaluate Random Forest
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_report = classification_report(y_test, y_pred_rf, zero_division=0)

# Train and evaluate Logistic Regression
lr_classifier.fit(X_train, y_train)
y_pred_lr = lr_classifier.predict(X_test)
lr_accuracy = accuracy_score(y_test, y_pred_lr)
lr_report = classification_report(y_test, y_pred_lr, zero_division=0)

# Display the results
dt_accuracy, rf_accuracy, lr_accuracy, dt_report, rf_report, lr_report


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.3333333333333333,
 0.46,
 0.3,
 '              precision    recall  f1-score   support\n\n           0       0.33      0.31      0.32        48\n           1       0.39      0.34      0.36        53\n           2       0.29      0.35      0.32        49\n\n    accuracy                           0.33       150\n   macro avg       0.34      0.33      0.33       150\nweighted avg       0.34      0.33      0.33       150\n',
 '              precision    recall  f1-score   support\n\n           0       0.46      0.52      0.49        48\n           1       0.48      0.38      0.42        53\n           2       0.44      0.49      0.47        49\n\n    accuracy                           0.46       150\n   macro avg       0.46      0.46      0.46       150\nweighted avg       0.46      0.46      0.46       150\n',
 '              precision    recall  f1-score   support\n\n           0       0.29      0.33      0.31        48\n           1       0.25      0.13      0.17        53\n         

In [ ]:
from sklearn.ensemble import VotingClassifier

# Create a Voting Classifier with Decision Tree, Random Forest, and Logistic Regression
voting_classifier = VotingClassifier(
    estimators=[
        ('dt', dt_classifier),
        ('rf', rf_classifier),
        ('lr', lr_classifier)
    ],
    voting='hard'  # 'hard' voting means majority voting; 'soft' would average probabilities
)

# Train the Voting Classifier
voting_classifier.fit(X_train, y_train)

# Predict and evaluate the Voting Classifier
y_pred_voting = voting_classifier.predict(X_test)
voting_accuracy = accuracy_score(y_test, y_pred_voting)
voting_report = classification_report(y_test, y_pred_voting, zero_division=0)

voting_accuracy, voting_report


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.4,
 '              precision    recall  f1-score   support\n\n           0       0.38      0.50      0.43        48\n           1       0.43      0.25      0.31        53\n           2       0.40      0.47      0.43        49\n\n    accuracy                           0.40       150\n   macro avg       0.41      0.40      0.39       150\nweighted avg       0.41      0.40      0.39       150\n')

Preprocess Text Data (NLP with TF-IDF on Job Titles and Skills)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Fill NaNs in text columns to avoid errors during vectorization
df['Job_Title'] = df['Job_Title'].fillna('')
df['Required_Skills'] = df['Required_Skills'].fillna('')

# Combine Job Title and Skills to create a single column for NLP
df['job_skills_combined'] = df['Job_Title'] + ' ' + df['Required_Skills']

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Adjust max_features as needed
tfidf_features = tfidf_vectorizer.fit_transform(df['job_skills_combined'])

# Convert TF-IDF features to a DataFrame and merge with original data
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
df = pd.concat([df.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)


In [ ]:
# Example: Dictionary of job titles and associated automation risks
automation_risk_map = {
    'Data Scientist': 'Low',
    'Customer Service Representative': 'High',
    'Software Engineer': 'Low',
    # Add more mappings as available
}

# Map the automation risk to each job title in the dataset
df['Job_Automation_Risk'] = df['Job_Title'].map(automation_risk_map)

# Fill missing values with 'Unknown' or 'Medium' as a default
df['Job_Automation_Risk'] = df['Job_Automation_Risk'].fillna('Medium')

# Convert risk categories to numerical values for model compatibility
risk_map = {'Low': 0, 'Medium': 1, 'High': 2}
df['Job_Automation_Risk'] = df['Job_Automation_Risk'].map(risk_map)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

X = pd.get_dummies(df.drop(columns=['Automation_Risk', 'Job_Title', 'Required_Skills', 'job_skills_combined']), drop_first=True)

# Continue with model training and evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model (Random Forest)
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)

# Display results
print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Accuracy: 0.35333333333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.34      0.26      0.30        53
           1       0.41      0.53      0.46        49
           2       0.28      0.27      0.28        48

    accuracy                           0.35       150
   macro avg       0.35      0.36      0.35       150
weighted avg       0.35      0.35      0.35       150



In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# Proceed with train-test split and model fitting as before
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Use XGBoost model
xgb_classifier = XGBClassifier(eval_metric='mlogloss', random_state=42)
grid_search = GridSearchCV(xgb_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Evaluate
y_pred_xgb = grid_search.best_estimator_.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
report_xgb = classification_report(y_test, y_pred_xgb, zero_division=0)

print("XGBoost Accuracy:", accuracy_xgb)
print("Classification Report:\n", report_xgb)



XGBoost Accuracy: 0.35333333333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.42      0.25      0.31        53
           1       0.34      0.43      0.38        49
           2       0.33      0.40      0.36        48

    accuracy                           0.35       150
   macro avg       0.36      0.36      0.35       150
weighted avg       0.37      0.35      0.35       150



In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define individual classifiers
xgb = XGBClassifier(learning_rate=0.1, max_depth=5, n_estimators=200, eval_metric='mlogloss', random_state=42)
rf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
lr = LogisticRegression(max_iter=200, random_state=42)

# Create a Voting Classifier
voting_clf = VotingClassifier(estimators=[('xgb', xgb), ('rf', rf), ('lr', lr)], voting='soft')
voting_clf.fit(X_train, y_train)

# Evaluate ensemble
y_pred = voting_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)

print("Ensemble Voting Classifier Accuracy:", accuracy)
print("Classification Report:\n", report)


Ensemble Voting Classifier Accuracy: 0.37333333333333335
Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.23      0.28        53
           1       0.38      0.57      0.46        49
           2       0.36      0.33      0.35        48

    accuracy                           0.37       150
   macro avg       0.37      0.38      0.36       150
weighted avg       0.37      0.37      0.36       150



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

# Load the dataset
df = pd.read_csv('ai_job_market_insights.csv')

# Encode categorical columns
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save the encoder for each column

# Define features and target variable
X = df.drop('Automation_Risk', axis=1)
y = df['Automation_Risk']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Train the XGBoost classifier
xgb_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_xgb = xgb_classifier.predict(X_test)

# Evaluate the model
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
xgb_report = classification_report(y_test, y_pred_xgb, zero_division=0)

# Display the results
xgb_accuracy, xgb_report


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:42:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


(0.4266666666666667,
 '              precision    recall  f1-score   support\n\n           0       0.39      0.40      0.39        48\n           1       0.45      0.36      0.40        53\n           2       0.44      0.53      0.48        49\n\n    accuracy                           0.43       150\n   macro avg       0.43      0.43      0.42       150\nweighted avg       0.43      0.43      0.42       150\n')

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

# Load the dataset
df = pd.read_csv('ai_job_market_insights.csv')

# Encode categorical columns
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save the encoder for each column

# Define features and target variable
X = df.drop('Automation_Risk', axis=1)
y = df['Automation_Risk']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(eval_metric='mlogloss', random_state=42)

# Set parameters for Grid Search
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'subsample': [0.5, 0.7, 1.0]
}

# Perform Grid Search
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Use best parameters to create a new optimized model
best_params = grid_search.best_params_
optimized_xgb_classifier = xgb.XGBClassifier(**best_params, eval_metric='mlogloss', random_state=42)

# Train the optimized model
optimized_xgb_classifier.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred_optimized = optimized_xgb_classifier.predict(X_test)
optimized_accuracy = accuracy_score(y_test, y_pred_optimized)
optimized_report = classification_report(y_test, y_pred_optimized, zero_division=0)

# Display the results
print("Optimized Model Accuracy:", optimized_accuracy)
print(optimized_report)


Best Parameters: {'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.5}
Best Score: 0.3628571428571428
Optimized Model Accuracy: 0.38666666666666666
              precision    recall  f1-score   support

           0       0.38      0.40      0.39        48
           1       0.38      0.32      0.35        53
           2       0.40      0.45      0.42        49

    accuracy                           0.39       150
   macro avg       0.39      0.39      0.39       150
weighted avg       0.39      0.39      0.38       150

